Exemplos *Classe PerfilILam*
-----------------------
Este notebook apresenta exemplos de utilização da classe *PerfilILam* que representa o perfil metálico do tipo *I laminado*. Estes exemplos tem por objetivo mostrar como utilizar a biblioteca Metalpy, apresentar algumas funcionalidades e servir como validação dos métodos implementados.

Os exemplos apresentados são exemplos já resolvidos obtidos de livros e manuais das seguintes referências:

[1] PFEIL, WALTER. **Estruturas de aço: dimensionamento prático.** 9ª Edição. Rio de Janeiro. LTC, 2022.

[2] AMERICAN INSTITUTE OF STEEL CONSTRUCTION. AISC 360: **Companion to the AISC steel construction manual - Volume 1: Design Examples.** V15.1, Chicago. Illinois, 2015.

-----------------------------------------------------------------------------------------------------------------------------

### Exemplo 1

Este exemplo é o exercícico 6.5.1.a) dos problemas resolvidos do capitulo 6 de [1], o exercicio trata da obtenção do momento fletor resistênte de um perfil I laminado W530X85. Considera-se o perfil contido lateralmente em toda a sua extensão. Aço MR250.

In [1]:
from metalpy.perfis import PerfilILam
from metalpy.material import Aco
import pandas as pd

#Definindo o aço do tipo MR250 com as propriedades em kN/cm²
MR250 = Aco(20000, 0.3, 25, 30, 0.6)

#Definindo um perfil I laminado do tipo W com as propriedades em cm
W530X85 = PerfilILam('W530X85', MR250, und='cm', norma='NBR8800')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [2]:
#Respostas do exercicio
r_esb_mesa = 5 # Esbeltez da mesa
r_esb_alma = 46.4 # Esbeltez da alma
r_Zx = 2100 # Módulo plástico
r_Mrdx = 47730 #Momento resistênte

#Respostas com Metalpy
mp_esb_mesa = W530X85.esb_mesa
mp_esb_alma =  W530X85.esb_alma
mp_Zx = W530X85.Zx
# Obtendo o momento fletor resistente em relação ao eixo X(maior inércia)
Mrdx, dados_Mrd = W530X85.Mrdx(Lb=0, Cb=1, data=True)

columns = 'Referência Metalpy'.split()
index = ['bf/2tf', 'h/tw', 'Zx (cm³)', 'Mrdx(kN.cm)']
data = zip([r_esb_mesa, r_esb_alma, r_Zx, r_Mrdx], [ mp_esb_mesa, mp_esb_alma, mp_Zx, Mrdx])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
bf/2tf,5.0,5.040000
h/tw,46.4,46.300000
Zx (cm³),2100.0,2110.000000
Mrdx(kN.cm),47730.0,47954.545455


In [3]:
# Verificação
tol = 0.02 #tolerência de 2% de erro

assert min(r_Mrdx, Mrdx)/ max(r_Mrdx, Mrdx) >= 1 - tol, 'Valor do momento resistênte excede o tolerência'

-------------------------------------------------------------------------------------------------------------------------
Exemplo 2
---------------
Exemplo E.1D de [2]. Determinação da força axial de compressão resistênte de uma coluna formada por um perfil W19X90 (equivalente ao W360X134 em mm e kg/m) através dos critérios de cálculo da AISC360-16. A coluna é constituida de aço ASTM A992 e comprimento 15ft (457.2 cm). As parâmetros geométricos do perfil estão apresentados abaixo:

* Ag = 26.5 in²
* rx = 6.14 in
* ry = 3.70 in
* bf/2tf = 10.2
* h/tw = 25.9

*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [4]:
from metalpy.perfis import PerfilI
from metalpy.normas import AISC360
from metalpy.material import Aco

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A992 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil W360X134 em unidades de cm
W360X134 = PerfilILam('W360X134', A992, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [5]:
#Respostas do exercicio
r_A = 26.5 # Área total
r_esb_mesa = 10.2 # Esbeltez da mesa
r_esb_alma =  25.9 # Esbeltez da alma
r_ry = 3.7 # Raio de giração em Y
r_rx = 6.14 # Raio de giração em X
r_Fe = 83.3 # Tensão critica de flambagem
r_Fcr = 38.9 # Tensão crítica
r_Ncrd = 927 # Força axial resistênte de cálculo pela AISC

#Respostas com Metalpy
mp_A =  W360X134.A * cm_to_in ** 2
mp_esb_mesa = W360X134.esb_mesa
mp_esb_alma = W360X134.esb_alma
mp_ry = W360X134.ry * cm_to_in
mp_rx = W360X134.rx * cm_to_in


# Calculando a força axial resistênte de cálculo através do método Ncrd e obtendo os dados do cálculo
Lx = 30 * ft_to_cm
Ly = 15 * ft_to_cm
Lz = 1
Ncrd, dados = W360X134.Ncrd(Lx, Ly, 1, data=True)
mp_Ncrd = Ncrd * kN_to_kips
mp_Fe = dados.Fe * kN_cm2_to_ksi
mp_Fcr = dados.Fcr * kN_cm2_to_ksi

columns = 'Referência Metalpy'.split()
index = ['A (in²)', 'bf/2tf', 'h/tw', 'ry (in)', 'rx (in)', 'Fe (ksi)', 'Fcr (ksi)', 'Ncrd (kips)']
data = zip([r_A, r_esb_mesa, r_esb_alma, r_ry, r_rx, r_Fe, r_Fcr, r_Ncrd], [ mp_A, mp_esb_mesa, mp_esb_alma, mp_ry, mp_rx, mp_Fe, mp_Fcr, mp_Ncrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
A (in²),26.50,26.545356
bf/2tf,10.20,10.200000
h/tw,25.90,25.900000
ry (in),3.70,3.702428
rx (in),6.14,6.145325
Fe (ksi),83.30,83.276512
Fcr (ksi),38.90,38.889389
Ncrd (kips),927.00,928.462721


In [6]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Ncrd, mp_Ncrd)/ max(r_Ncrd, mp_Ncrd) >= 1 - tol, 'Valor da força axial resistênte excede a tolerância'

-------------------------------------------------------------------------------------------------------------------------
Exemplo 3
---------------
Exemplo F.1 de [2]. Determinação do momento fletor resistênte de cálculo de uma viga formada por um perfil W18X50 (equivalente ao W460X74 em mm e kg/m) através dos critérios de cálculo da AISC360-16. A coluna é constituida de aço ASTM A992 e comprimento 15ft (457.2 cm). Considera-se três casos de ctravamento lateral:

* 1A - Viga contida em todo seu comprimento (Lb=0)
* 2A - Viga travada a cada 1/3 do seu comprimento (Lb=11.67 in (355.6 cm))
* 3A - Viga travada no centro (Lb=17.5 in (533.4 cm))

*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [7]:
from metalpy.perfis import PerfilILam
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm)
kNcm_to_kipin = kN_to_kips * cm_to_in 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A992 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil W460X74
W460X74 = PerfilILam('W460X74', A992, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

Situação 1 - Viga travada em todo comprimento (Lb=0)

In [8]:
#Respostas do exercicio
r_Zx = 101 # Módulo plástico
r_Mrd1 = 379

#Respostas com Metalpy
mp_Zx =  W460X74.Zx * cm_to_in ** 3

# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Lb = 0
Cb = 1
Mrd= W460X74.Mrdx(Lb = Lb, Cb = Cb)
mp_Mrd1 = Mrd * kNcm_to_kipft

columns = 'Referência Metalpy'.split()
index = ['Zx (in²)', 'Mrd (kip.ft)']
data = zip([r_Zx, r_Mrd1], [ mp_Zx, mp_Mrd1])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
Zx (in²),101,101.530553
Mrd (kip.ft),379,380.189717


In [9]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Mrd1, mp_Mrd1)/ max(r_Mrd1, mp_Mrd1) >= 1 - tol, 'Valor do moemnto fletor resistente excede a tolerância'

Situação 2 - Viga travada a cada 1/3 do comprimento (Lb = 11.67 in (355.6 cm ), Cb = 1.01)

In [10]:
#Respostas do exercicio
r_ry = 1.65 # Raio de giração em Y
r_Sx = 88.9 # Módulo elástico
r_J= 1.24 # Inércia a torção
r_Lp = 69.9 # Comprimento limite plástico
r_Lr = 203 # Comprimento limite para o inicio de escoamento
r_Mrd2= 305 # Momento resistente de cálculo

#Respostas com Metalpy
mp_ry =  W460X74.ry * cm_to_in
mp_Sx = W460X74.Wx * cm_to_in ** 3
mp_J = W460X74.J * cm_to_in ** 4

# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Lb = 11.67 * ft_to_cm
Cb = 1.01
Mrd, dados_Mrd = W460X74.Mrdx(Lb = Lb, Cb = Cb, data=True)
mp_Mrd2 = Mrd * kNcm_to_kipft
mp_Lp = dados_Mrd.FLT.Lp * cm_to_in
mp_Lr = dados_Mrd.FLT.Lr * cm_to_in

columns = 'Referência Metalpy'.split()
index = ['ry (in)', 'Sx (in³)', 'J (in³)', 'Lp (in)', 'Lr (in)', 'Mrd (kip.ft)']
data = zip([r_ry, r_Sx, r_J, r_Lp, r_Lr, r_Mrd2], [ mp_ry, mp_Sx, mp_J, mp_Lp, mp_Lr, mp_Mrd2])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
ry (in),1.65,1.653675
Sx (in³),88.90,89.297957
J (in³),1.24,1.243468
Lp (in),69.90,70.093372
Lr (in),203.00,203.375351
Mrd (kip.ft),305.00,306.422292


In [11]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Mrd2, mp_Mrd2)/ max(r_Mrd2, mp_Mrd2) >= 1 - tol, 'Valor do moemnto fletor resistente excede a tolerância'

Situação 3 - Viga travada no centro (Lb = 17.5 in (533.4 cm ), Cb = 1.30)

In [12]:
#Respostas do exercicio
r_ry = 1.65 # Raio de giração em Y
r_Sx = 88.9 # Módulo elástico
r_J= 1.24 # Inércia a torção
r_Lp = 69.90 # Comprimento limite plástico
r_Lr = 203 # Comprimento limite para o inicio de escoamento
r_Mrd3= 288 # Momento resistente de cálculo

#Respostas com Metalpy
mp_ry =  W460X74.ry * cm_to_in
mp_Sx = W460X74.Wx * cm_to_in ** 3
mp_J = W460X74.J * cm_to_in ** 4

# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Lb = 17.5 * ft_to_cm
Cb = 1.3
Mrd, dados_Mrd = W460X74.Mrdx(Lb = Lb, Cb = Cb, data=True)
mp_Mrd3 = Mrd * kNcm_to_kipft
mp_Lp = dados_Mrd.FLT.Lp * cm_to_in
mp_Lr = dados_Mrd.FLT.Lr * cm_to_in

columns = 'Referência Metalpy'.split()
index = ['ry (in)', 'Sx (in³)', 'J (in³)', 'Lp (in)', 'Lr (in)', 'Mrd (kip.ft)']
data = zip([r_ry, r_Sx, r_J, r_Lp, r_Lr, r_Mrd3], [ mp_ry, mp_Sx, mp_J, mp_Lp, mp_Lr, mp_Mrd3])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
ry (in),1.65,1.653675
Sx (in³),88.90,89.297957
J (in³),1.24,1.243468
Lp (in),69.90,70.093372
Lr (in),203.00,203.375351
Mrd (kip.ft),288.00,288.413814


In [13]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Mrd3, mp_Mrd3)/ max(r_Mrd3, mp_Mrd3) >= 1 - tol, 'Valor do moemnto fletor resistente excede a tolerância'

-------------------------------------------------------------------------------------------------------------------------
Exemplo 4
---------------
Exemplo F.3 de [2]. Determinação do momento fletor resistênte de cálculo de uma viga formada pelo perfil de mesas esbeltas W21X48 (equivalente ao W530X72 em mm e kg/m) através dos critérios de cálculo da AISC360-16. A viga é constituida de aço ASTM A992 e apresenta travamento lateral continuo em toda a sua extensão.


*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [14]:
from metalpy.perfis import PerfilILam
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm)
kNcm_to_kipin = kN_to_kips * cm_to_in 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A992 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil W530X72
W530X72 = PerfilILam('W530X72', A992, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [15]:
#Respostas do exercicio
r_Sx = 93 # Módulo elástico
r_Zx= 107 # Módulo Plástico
r_esb_mesa = 9.47 # Exbeltez da mesa
r_elp = 9.15 # parâmetro de esbeltez limite de plastificação
r_elr = 24.1 # parâmetro de esbeltez limite de inicio de escoamento
r_Mrd = 398 # Momento resistente

#Respostas com Metalpy
mp_Sx = W530X72.Wx * cm_to_in ** 3
mp_Zx = W530X72.Zx * cm_to_in ** 3
mp_esb_mesa = W530X72.esb_mesa

# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Lb = 0
Cb = 1
Mrd, dados_Mrd = W530X72.Mrdx(Lb = Lb, Cb = Cb, data=True)
mp_Mrd = Mrd * kNcm_to_kipft
mp_elp = dados_Mrd.FLM.elpf
mp_elr = dados_Mrd.FLM.elrf

columns = 'Referência Metalpy'.split()
index = ['Sx (in³)', 'Zx (in³)', 'bf/2tf', 'elp', 'elr', 'Mrd (kip.ft)']
data = zip([r_Sx, r_Zx, r_esb_mesa, r_elp, r_elr, r_Mrd], [ mp_Sx, mp_Zx, mp_esb_mesa, mp_elp, mp_elr, mp_Mrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
Sx (in³),93.00,92.967736
Zx (in³),107.00,107.035222
bf/2tf,9.47,9.470000
elp,9.15,9.151612
elr,24.10,24.083189
Mrd (kip.ft),398.00,397.452237


In [16]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Mrd, mp_Mrd)/ max(r_Mrd, mp_Mrd) >= 1 - tol, 'Valor do moemento fletor resistente excede a tolerância'

-------------------------------------------------------------------------------------------------------------------------
Exemplo 5
---------------
Exemplo F.3 de [2]. Determinação do momento fletor resistênte de cálculo de uma viga formada pelo perfil W12X58 (equivalente ao W310X86 em mm e kg/m), fletido em relação ao eixo de menor inércia (eixo Y), através dos critérios de cálculo da AISC360-16. A viga é constituida de aço ASTM A992, comprimento de 15ft (457.2 cm) e apresenta travamento somente nas extremidades de apoio.


*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [17]:
from metalpy.perfis import PerfilILam
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm)
kNcm_to_kipin = kN_to_kips * cm_to_in 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A992 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil W310X86
W310X86 = PerfilILam('W310X86', A992, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [18]:
#Respostas do exercicio
r_Sy = 21.4 # Módulo elástico
r_Zy= 32.5 # Módulo Plástico
r_Iy = 107 # Momento de inércia
r_Mrd = 122

#Respostas com Metalpy
mp_Sy = W310X86.Wy * cm_to_in ** 3
mp_Zy = W310X86.Zy * cm_to_in ** 3
mp_Iy = W310X86.Iy * cm_to_in ** 4

# Calculando o momento resistente de cálculo e obtendo os dados do cálculo
Cb = 1
Lb = 15 * ft_to_cm
Mrd, dados_Mrd = W310X86.Mrdy(Lb = Lb, Cb = Cb, data=True)
mp_Mrd = Mrd * kNcm_to_kipft

columns = 'Referência Metalpy'.split()
index = ['Sy (in³)', 'Zy (in³)', 'Iy', 'Mrd (kip.ft)']
data = zip([r_Sy, r_Zy, r_Iy, r_Mrd], [ mp_Sy, mp_Zy, mp_Iy, mp_Mrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
Sy (in³),21.4,21.468207
Zy (in³),32.5,32.599870
Iy,107.0,107.237060
Mrd (kip.ft),122.0,122.072963


In [19]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Mrd, mp_Mrd)/ max(r_Mrd, mp_Mrd) >= 1 - tol, 'Valor do momento resistente resistênte excede a tolerância'

-------------------------------------------------------------------------------------------------------------------------
Exemplo 6
---------------
Exemplo G.1 de [2]. Determinação da força cortante resistente de calculo de uma viga formado por um perfil W24X62 (equivalente ao W610X92 em mm e kg/m), fletida em relação ao eixo de maior inércia (eixo X), através dos critérios de cálculo da AISC360-16. A viga é constituida de aço ASTM A992.

*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [20]:
from metalpy.perfis import PerfilILam
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm)
kNcm_to_kipin = kN_to_kips * cm_to_in 

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A992 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil W610X92
W610X92 = PerfilILam('W610X92', A992, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [21]:
#Respostas do exercicio
r_d = 23.7 # Módulo elástico
r_tw= 0.430 # Módulo Plástico
r_Aw = 10.2 # Momento de inércia
r_Vrd = 306 # Força cortante resistente de cálculo

#Respostas com Metalpy
mp_d = W610X92.d * cm_to_in
mp_tw = W610X92.tw * cm_to_in
mp_Aw = W610X92.Awy * cm_to_in ** 2

# Calculando a força resistente de calculo no sentindo do eixo Y
mp_Vrd = W610X92.Vrdy() * kN_to_kips

columns = 'Referência Metalpy'.split()
index = ['d (in)', 'tw (in)', 'Aw (in²)', 'Vrd (kip)']
data = zip([r_d, r_tw, r_Aw, r_Vrd], [ mp_d, mp_tw, mp_Aw, mp_Vrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
d (in),23.70,23.718800
tw (in),0.43,0.429460
Aw (in²),10.20,10.186276
Vrd (kip),306.00,305.378861


In [22]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Vrd, mp_Vrd)/ max(r_Vrd, mp_Vrd) >= 1 - tol, 'Valor da força cortante resistênte excede a tolerância'

-------------------------------------------------------------------------------------------------------------------------
Exemplo 7
---------------
Exemplo H.1 de [2]. Verificação a flexo-tração de uma barra formada por um perfil W14X82 (Equivalente ao W360X122 em mm e kg/m) de acordo com os critérios da AISC360-16. A barra apresenta um comprimento de $30 ft (914.4 cm)$ e está submetida a uma carga permanente de tração de $29 kips (128.89 kN)$ e váriável de $87 kips (386.84 kN)$. Também está submetida a momento nas duas direções devido a cargas lineramente distríbuidas, os valores dos momentos estão apresentados abaixo:

* $ M_{sxp} = 32 kip.ft $   $ (4334.93 kN.cm)$ (Momento em X devido a carga permanente)
* $ M_{sxa} = 96 kip.ft $   $ (13004.8 kN.cm)$ (Momento em X devido a carga variável)
* $ M_{syp} = 11.3 kip.ft $   $ (1530.77 kN.cm)$ (Momento em Y devido a carga permanente)
* $ M_{sya} = 33.8 kip.ft $   $ ( 4578.77 kN.cm)$ (Momento em Y devido a carga variável)

As solicitações de cálculo estão apresentadas a seguir:

* $ N_{tsd} = 174 kips$  $(773.33 kN)$
* $ M_{xsd} = 192 kip.ft$  $(26009.6 kN.cm)$
* $ M_{ysd} = 67.6 kip.ft$  $(9157.55 kN.cm)$

O aço que compõe a barra é o ASTM A992 que aprsenta $fy = 50 ksi (34.5 kN/cm^2)$ e $ fu = 65 ksi (44.8 kN/cm²) $. No exemplo foi utilizado um coeficiente Cb igual à  1.41.

*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [23]:
from metalpy.perfis import PerfilILam
from metalpy.normas import AISC360
from metalpy.material import Aco

import pandas as pd

#Unidades para conversão
cm_to_in = 0.394
ft_to_cm = 30.48
cm_to_ft = 1 / ft_to_cm 
kN_to_kips = 0.225 
kNcm_to_kipft = kN_to_kips * (1/ft_to_cm) 


#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A992 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil W360X122 com as propriedades em cm
W360X122 = PerfilILam('W360X122', A992, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [24]:
#Respostas do exercicio
r_A = 24  # Área total
r_Wx = 123 # Módulo elástico em relação ao eixo X  
r_Zx = 139 # Módulo plástico em relação ao eixo X
r_Wy = 29.3 # Módulo elástico em relação ao eixo Y
r_Zy = 44.8 # Módulo plástico em relação ao eixo Y
r_Iy = 148 # Momento de inércia  em relação ao eixo Y
r_Lp = 8.76 # Comprimento livre limite de plastificação
r_Lr = 33.2 # Comprimento livre limite para início de escoamento
r_Ntrd = 1080 # Força de tração resistente de cálculo
r_Mrdx = 492 #Momento resistênte de cálculo em relação ao eixo X
r_Mrdy = 168 # Momento resistênte de cálculo em relação ao eixo Y
r_verif = 0.873 # Verificação à flexo-tração

#Respostas com Metalpy
mp_A =  W360X122.A * cm_to_in ** 2
mp_Wx =  W360X122.Wx * cm_to_in ** 3
mp_Zx =  W360X122.Zx * cm_to_in ** 3
mp_Wy =  W360X122.Wy * cm_to_in ** 3
mp_Zy =  W360X122.Zy * cm_to_in ** 3
mp_Iy =  W360X122.Iy * cm_to_in ** 4

# Calculando a força de tração resistente de cálculo de acordo com o ELU de escoamento da seção bruta
mp_Ntrd = W360X122.Ntrd_brt() * kN_to_kips

# Calculando o momento resistênte em X e obtendo os dados de cálculo
Cbx = 1.41
Lbx = 30 * ft_to_cm

Mrdx, dados_Mrdx = W360X122.Mrdx(Lb = Lbx, Cb = Cbx, data=True)
mp_Mrdx = Mrdx * kNcm_to_kipft
mp_Lp = dados_Mrdx.FLT.Lp * cm_to_ft
mp_Lr = dados_Mrdx.FLT.Lr * cm_to_ft

# Calculando o momento resistênte em Y
mp_Mrdy = W360X122.Mrdy(Lb = 0, Cb=1) * kNcm_to_kipft

# Verificação a flexo-tração
Nsd = 174 / kN_to_kips
Msdx = 192 / kNcm_to_kipft
Msdy = 67.6 / kNcm_to_kipft

mp_verif = W360X122.verif_NM(Nsd, Msdx, Msdy, Lbx= 30 * ft_to_cm, Cb=1.41, klx=1, kly=Lbx, klz=1)

columns = 'Referência Metalpy'.split()
index = ['A(in²)', 'Wx(in³)', 'Zx(in³)', 'Wy(in³)', 'Zy(in³)', 'Iy(in4)', 'Lp(ft)', 'Lr(ft)', 'Ntrd(kips)', 'Mrdx(kips.ft)', 'Mrdy(kips.ft)', 'NM']
data = zip([r_A, r_Wx, r_Zx, r_Wy, r_Zy, r_Iy, r_Lp, r_Lr, r_Ntrd, r_Mrdx, r_Mrdy, r_verif], [ mp_A, mp_Wx, mp_Zx, mp_Wy, mp_Zy, mp_Iy, mp_Lp, mp_Lr, mp_Ntrd, mp_Mrdx, mp_Mrdy, mp_verif])

df = pd.DataFrame(data,index=index,columns=columns)
df

c:\users\55519\documents\ufrgs\tcc\código\projeto de tcc - verificações de estrututas metálicas\metalpy\metalpy\normas\_aisc.py:67: UserWarning: Item D1: A AISC360-16 recomenda que índice de esbeltez de barras tracionadas não supere o valor de 300
  warn('Item D1: A AISC360-16 recomenda que índice de esbeltez de barras tracionadas não '


,Referência,Metalpy
A(in²),24.000,24.061580
Wx(in³),123.000,123.549228
Zx(in³),139.000,139.451604
Wy(in³),29.300,29.358232
Zy(in³),44.800,44.893630
Iy(in4),148.000,148.445009
Lp(ft),8.760,8.766701
Lr(ft),33.200,33.020622
Ntrd(kips),1080.000,1082.029095
Mrdx(kips.ft),492.000,491.455393
